# This notebook is used to generate exhaustive epistatic features using genotypic data. 

Sample datasets are generated using GAMETES:

Dataset 1 - Continuous Phenotyoe
Dataset 2 - Discrete Phenotype

### Step 1: Import all the required packages

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from itertools import combinations, product
from sklearn.preprocessing import MinMaxScaler
import numpy as np

### Step 2: Define the function to generate the epistatic features exhaustively

In [63]:
'''Function definition to generate the PAGER epistatis encodings using the training genotypes and phenotype. 
    
    Input Parameters

    ----------------
    genotypes_train : pandas dataframe
        A dataframe containing the training genotypes of the SNPs. The dataframe should have the SNPs as columns and the samples as rows.
    phenotype : pandas series
        A series containing the training phenotype of the samples. The series should have the samples as index and the phenotypes as values.
    order : int
        The order of the epistasis to be considered. For example, if order=2, then the function will consider all possible two-way epistasis.

    Returns

    ----------------
    pager_epistatis_mapping : pandas dataframe
        A dataframe containing the PAGER epistatic encodings for each SNP combination. The dataframe will have the SNP combination and the corresponding PAGER epistatic encoding.
   all_missing_multilocus : pandas dataframe
        A dataframe containing all the missing multilocus genotypes for all SNP combinations. The dataframe will have the SNP combination and the corresponding missing multilocus genotypes.
    '''

def get_pager_epistatis_mapping(genotypes_train, phenotype_train, order):
    # All possible SNP/feature combinations for order-way epistasis
    snp_combinations = list(combinations(genotypes_train.columns, order))

    # All possible genotypic (0,1,2) combinations for order-way epistasis - used to find missing multilocus genotypes
    unique_genotypes = genotypes_train.stack().unique()
    possible_genotypic_combinations = list(product(unique_genotypes, repeat=order))

    # Create a list to hold missing multilocus dataframes for all iterations
    missing_multilocus_list = []

    pager_epistasis_mapping = []  # List to store all aggregations for each SNP combination - final output of the function

    # Loop through all SNP combinations
    for snp_combination in snp_combinations:
        # Make a DataFrame with the SNP combination and the phenotype
        snp_combination_df_train = genotypes_train[list(snp_combination)].copy()
        snp_combination_df_train.loc[:, 'Phenotype'] = phenotype_train

        snp_columns = list(snp_combination_df_train.columns[:-1])

        # Create a DataFrame with all possible genotype combinations
        all_combinations_df = pd.DataFrame(possible_genotypic_combinations, columns=snp_columns)

        # Get the mean phenotype per SNP combination
        geno_aggregations_train = snp_combination_df_train.groupby(snp_columns).agg(
            mean_phenotype=('Phenotype', 'mean')
        ).reset_index()  # Reset index to make the groupby columns as columns

        anchor_mean = geno_aggregations_train['mean_phenotype'].iloc[
            geno_aggregations_train['mean_phenotype'].first_valid_index()]  # The first valid genotype combination is considered as the anchor, it starts from (0,0) for 2-way epistasis

        # Calculate the relative distance of the mean phenotype from the anchor
        geno_aggregations_train['rel_dist'] = geno_aggregations_train['mean_phenotype'] - anchor_mean

        # Min-Max normalization on rel_dist (relative distance)
        scaler_train = MinMaxScaler()
        geno_aggregations_train['normalized_rel_dist'] = scaler_train.fit_transform(
            geno_aggregations_train['rel_dist'].values.reshape(-1, 1)
        )

        # Finding the missing multilocus genotypes
        missing_combinations_df_train = pd.merge(all_combinations_df, geno_aggregations_train, on=snp_columns, how='left')
        missing_combinations_train = missing_combinations_df_train[missing_combinations_df_train['mean_phenotype'].isnull()]

        # Reset index
        missing_combinations_train = missing_combinations_train.reset_index(drop=True)

        # Convert missing combinations to comma-separated string of tuples
        missing_multilocus_genotypes_train = missing_combinations_train[snp_columns].apply(tuple, axis=1)

        # Create a DataFrame for the SNP combination with all missing genotypes
        if len(missing_multilocus_genotypes_train) > 0:
            df_to_add = pd.DataFrame([{
                **dict(zip([f'SNP{i+1}' for i in range(order)], snp_combination)),
                'Missing_Multilocus_Genotype': ','.join(map(str, missing_multilocus_genotypes_train))
            }])
            missing_multilocus_list.append(df_to_add)

        # Store geno_aggregations for the current SNP combination
        pager_epistasis_mapping.append(geno_aggregations_train)

    # Concatenate all the missing multilocus dataframes
    all_missing_multilocus_df = pd.concat(missing_multilocus_list, ignore_index=True)

    return pager_epistasis_mapping, all_missing_multilocus_df


### Step 3: Define a function to take as input the original genotype data, the PAGER Epistatis mapping and order to create the new PAGER Epistatis dataset

In [60]:
def generate_pager_epistatis_dataset(genotypes, pager_epistatis_mapping, order):
    
    # DataFrame to store newly created PAGER epistatic dataset
    pager_epistatis_dataset = pd.DataFrame()

    # Dictionary to store the epistasis features before concatenating
    epistasis_features = {}

    # all possible SNP/feature combination for order-way epistasis
    snp_combinations = list(combinations(genotypes.columns, order))

    # Loop through each SNP combination and merge with corresponding geno_aggregations
    for snp_combination, pager_epistatis_mapping in zip(snp_combinations, pager_epistatis_mapping):
        snp_combination_genotypes = genotypes[list(snp_combination)]
        snp_columns = list(snp_combination_genotypes.columns)

        # Merge test data with normalized_rel_dist from training aggregation
        merged_df_test = pd.merge(snp_combination_genotypes, pager_epistatis_mapping, on=snp_columns, how='left')

        # Create a new column with the interaction feature for the test data
        col_name = '_'.join(snp_columns)
        #pager_epistatis_dataset[col_name] = merged_df_test['normalized_rel_dist']

        # Store the column in the dictionary
        epistasis_features[col_name] = merged_df_test['normalized_rel_dist']
    
    pager_epistatis_dataset = pd.concat(epistasis_features, axis=1)


    return pager_epistatis_dataset

### Step 4: Create the PAGER epistatic features using the above defined function - Dataset 1 (Continuous phenotype)

In [61]:
# read the genotypes and phenotype data
data = pd.read_csv('/Users/ghosha/Documents/VSCode Projects/PAGER/data/XOR_2way_cont.csv')
genotypes = data.iloc[:, :-1]
phenotype = data.iloc[:, -1]
order = 2

# Split the data into training and testing - the recommended split is 50% training and 50% testing
genotypes_train, genotypes_test, phenotype_train, phenotype_test = train_test_split(genotypes, phenotype, test_size=0.5, random_state=42)

# Generate the PAGER epistasis encodings using the training genotypes and phenotype
pager_epistasis_mapping,all_missing_multilocus = get_pager_epistatis_mapping(genotypes_train, phenotype_train, order)

# Generate the PAGER epistasis dataset using the test genotypes and the PAGER epistasis mapping
pager_epistatis_dataset = generate_pager_epistatis_dataset(genotypes_test, pager_epistasis_mapping, order)

# impute missing values with 0.5
pager_epistatis_dataset.fillna(0.5, inplace=True)

# print the PAGER epistasis dataset
print(pager_epistatis_dataset)
# save the PAGER epistasis dataset
pager_epistatis_dataset.to_csv('/Users/ghosha/Documents/VSCode Projects/PAGER/data/pager_epistatis_dataset.csv', index=False)

# print the missing multilocus genotypes
print(all_missing_multilocus)
# save the missing multilocus genotypes
all_missing_multilocus.to_csv('/Users/ghosha/Documents/VSCode Projects/PAGER/data/all_missing_multilocus.csv', index=False)

      1.281788173_10.84091208  1.281788173_7.129118847  \
0                    0.454190                 0.484567   
1                    0.473343                 1.000000   
2                    0.072273                 0.526460   
3                    0.373219                 0.526460   
4                    0.072273                 0.444915   
...                       ...                      ...   
2778                 0.072273                 0.444915   
2779                 0.349106                 0.382835   
2780                 0.541761                 0.526460   
2781                 0.541761                 0.444915   
2782                 0.373219                 0.526460   

      1.281788173_5.107167969  1.281788173_19.24321261  \
0                    0.222862                 0.642402   
1                    0.780433                 0.725770   
2                    0.444723                 0.311322   
3                    0.488342                 0.311322   
4            

### Step 5: Create the PAGER epistatic features using the above defined function - Dataset 2 (Discrete phenotype)

In [64]:
# read the genotypes and phenotype data
data = pd.read_csv('/Users/ghosha/Documents/VSCode Projects/PAGER/data/XOR_2way_disc.csv')
genotypes = data.iloc[:, :-1]
phenotype = data.iloc[:, -1]
order = 2

# Split the data into training and testing - the recommended split is 50% training and 50% testing
genotypes_train, genotypes_test, phenotype_train, phenotype_test = train_test_split(genotypes, phenotype, test_size=0.5, random_state=42)

# Generate the PAGER epistasis encodings using the training genotypes and phenotype
pager_epistasis_mapping,all_missing_multilocus = get_pager_epistatis_mapping(genotypes_train, phenotype_train, order)

# Generate the PAGER epistasis dataset using the test genotypes and the PAGER epistasis mapping
pager_epistatis_dataset = generate_pager_epistatis_dataset(genotypes_test, pager_epistasis_mapping, order)

# impute missing values with 0.5
pager_epistatis_dataset.fillna(0.5, inplace=True)

# print the PAGER epistasis dataset
print(pager_epistatis_dataset)
# save the PAGER epistasis dataset
pager_epistatis_dataset.to_csv('/Users/ghosha/Documents/VSCode Projects/PAGER/data/pager_epistatis_dataset.csv', index=False)

# print the missing multilocus genotypes
print(all_missing_multilocus)
# save the missing multilocus genotypes
all_missing_multilocus.to_csv('/Users/ghosha/Documents/VSCode Projects/PAGER/data/all_missing_multilocus.csv', index=False)

         N0_N1     N0_N2     N0_N3     N0_N4     N0_N5     N0_N6     N0_N7  \
0     0.848908  1.000000  0.199269  0.668466  0.428066  0.595156  0.572988   
1     0.754143  0.802856  0.164428  0.041588  0.297228  0.375868  0.331636   
2     0.775450  0.889243  0.199269  0.803534  0.077461  0.595156  0.572988   
3     0.615910  0.925346  0.119380  0.044444  0.000000  0.360406  0.000000   
4     0.775450  0.889243  0.199269  0.378164  0.428066  0.595156  0.393683   
...        ...       ...       ...       ...       ...       ...       ...   
2495  0.775450  0.910622  0.199269  0.668466  0.394925  0.490924  1.000000   
2496  1.000000  0.889243  0.199269  0.668466  0.394925  0.490924  0.393683   
2497  0.754143  0.802856  0.164428  1.000000  0.297228  0.375868  0.383011   
2498  0.754143  0.666632  0.164428  0.041588  0.533732  0.375868  0.331636   
2499  0.848908  0.889243  0.199269  0.378164  0.394925  0.490924  0.572988   

         N0_N8     N0_N9    N0_N10  ...   N15_N16   N15_N17  N1

### Step 6: User can use the newly generated dataset to run a statistical (p-value based) or machine learning method of ranking interactions to check for the top features